In [1]:
#!pip3 install sklearn
#!pip3 install plotly

## 2. PREPARACIÓN DE DATOS
Describa cómo ha transformado los datos cambiando las variables originales o creando nuevas variables. Si no muestra cómo transformó los datos, no podremos ver por qué su modelo funcionó bien o mal. Si transformó los datos o creó nuevas variables, explíquenos por qué.

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import plotly.graph_objects as go

![](../images/excel.png)

In [6]:
#Definición de funciones
def cambio_distribucion(X,opt):
    if opt == 'ambos':
        #Dataset completo
        x1 = df[X]
        x2 = imputed_df[X]
    elif opt == 'crash':
        #Dataset filtrado por coque
        x1 = df[df[trg[0]]==1][X]
        x2 = df2[df2[trg[0]]==1][X]
    elif opt == 'libre':
        #Dataset filtrado por libre de accidente
        x1 = df[df[trg[0]]==0][X]
        x2 = df2[df2[trg[0]]==0][X]
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=x1,name='Cruda'))
    fig.add_trace(go.Histogram(x=x2,name='Limpia'))
    fig.update_layout(barmode='stack')
    print(len(x1))
    print(len(x2))
    return fig
import plotly.figure_factory as ff
def plot_heatmap(df,variable):
    if variable == trg[0]:
        corr = df.drop(labels=trg[1],axis=1).corr()
    elif variable == trg[1]:
        corr = df[df[trg[0]]==1].drop(labels=trg[0],axis=1).corr()
    label_x,label_y = [],[]
    k = len(corr.index)-1
    for i in range(len(corr.index)):
        label_x.append(corr.index[i])
        label_y.append(corr.index[k])
        k-=1
    corr = np.asarray(corr)
    matrix,ax = [],[]
    for i in range(corr.shape[0]):
        fila = []
        k = corr.shape[1]-1
        for j in range(corr.shape[1]):
            fila.append(round(corr[i,k],2))
            k-=1
        matrix.append(fila)
    fig = ff.create_annotated_heatmap(z=matrix,x=label_x,y=label_y)
    return fig

In [9]:
#Lectura y definición de tipos de variables
df = pd.read_csv('../data/seguros_v2.csv')
#Variables Target
trg = ['TARGET_FLAG', 'TARGET_AMT']
#Variables Numericas
con = ['AGE','BLUEBOOK','CAR_AGE','CLM_FREQ','HOMEKIDS','HOME_VAL','INCOME','MVR_PTS','OLDCLAIM','TIF','TRAVTIME','YOJ']
#Variables Cateoricas
cat = ['CAR_TYPE','CAR_USE','EDUCATION','JOB','KIDSDRIV','MSTATUS','PARENT1','RED_CAR','REVOKED','SEX','URBANICITY']

In [8]:
## Limpieza Variables Categoricas

In [10]:
#Limpieza CAR_AGE, EXxistia un carro con -3 años este se paso a 0 años
df['CAR_AGE'] = [0 if i<=0 else i for i in df['CAR_AGE']]

In [11]:
#Eliminación de registros que superan el Q=995 para las valiables 'TARGET_AMT','BLUEBOOK','HOME_VAL','INCOME','OLDCLAIM','TRAVTIME' 
#Esto se traduce en una disminución de un 13.02% de la data, la eliminación de estos regstros trajo con sigo la eliminación de algunos registros nulos 
df2 = df.copy()
lis = ['TARGET_AMT','BLUEBOOK','HOME_VAL','INCOME','OLDCLAIM','TRAVTIME']
for i in lis:
    Q3 =df[i].quantile(0.995)
    df2 = df2[df2[i]<Q3]
df2 = df2.reset_index(drop=True)
print('Procentaje borrado de regitros es: {} - {} = {} ó {} '.format(df.shape[0],df2.shape[0],df.shape[0]-df2.shape[0],1-(df2.shape[0]/df.shape[0])))

Procentaje borrado de regitros es: 8161 - 7098 = 1063 ó 0.13025364538659479 


In [12]:
#Imputación de los datos faltantes para las categorias de Car_Age Yoj y Age
cols_to_impute = ['CAR_AGE','YOJ','AGE']

imputer = IterativeImputer()
imputed_df = pd.DataFrame(imputer.fit_transform(df2[cols_to_impute]))
imputed_df
imputed_df.columns = cols_to_impute
#imputed_df.index = reviews.index
for i in cols_to_impute:
    df2[[i]] = imputed_df[i]

In [13]:
#Registros nulos antes y despues de la limpieza anterior 
print('{:<12}  {:3}  {:3}'.format('Columna','Crudo','Limpio'))
for i in df.columns:
    print('{:<12}  {:6}  {:6}'.format(i,len(df[i])-df[i].count(),len(df2[i])-df2[i].count()   ))

Columna       Crudo  Limpio
TARGET_FLAG        0       0
TARGET_AMT         0       0
AGE                6       0
BLUEBOOK           0       0
CAR_AGE          510       0
CAR_TYPE           0       0
CAR_USE            0       0
CLM_FREQ           0       0
EDUCATION          0       0
HOMEKIDS           0       0
HOME_VAL         464       0
INCOME           445       0
JOB                0       0
KIDSDRIV           0       0
MSTATUS            0       0
MVR_PTS            0       0
OLDCLAIM           0       0
PARENT1            0       0
RED_CAR            0       0
REVOKED            0       0
SEX                0       0
TIF                0       0
TRAVTIME           0       0
URBANICITY         0       0
YOJ              454       0


In [14]:
df2.describe()

,TARGET_FLAG,TARGET_AMT,AGE,BLUEBOOK,CAR_AGE,CLM_FREQ,HOMEKIDS,HOME_VAL,INCOME,KIDSDRIV,MVR_PTS,OLDCLAIM,TIF,TRAVTIME,YOJ
count,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000,7098.000000
mean,0.262891,1264.691477,44.690798,15536.456748,8.291771,0.791772,0.729501,152950.384474,60384.789941,0.171739,1.691462,3825.674838,5.368132,33.202733,10.500702
std,0.440235,2975.398873,8.639334,8070.553718,5.532296,1.161488,1.122900,125296.245947,44451.079651,0.513266,2.144962,8233.538836,4.138559,15.353040,3.966176
min,0.000000,0.000000,16.000000,1500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000
25%,0.000000,0.000000,39.000000,9350.000000,4.000000,0.000000,0.000000,0.000000,27922.500000,0.000000,0.000000,0.000000,1.000000,22.000000,9.000000
50%,0.000000,0.000000,45.000000,14400.000000,8.000000,0.000000,0.000000,161034.000000,53544.000000,0.000000,1.000000,0.000000,4.000000,33.000000,11.000000
75%,1.000000,980.000000,51.000000,20667.500000,12.000000,2.000000,1.000000,237924.250000,84797.000000,0.000000,3.000000,4551.250000,7.000000,44.000000,13.000000
max,1.000000,32203.362274,81.000000,42170.000000,28.000000,5.000000,5.000000,544976.000000,234825.000000,4.000000,13.000000,46652.000000,25.000000,80.000000,23.000000


In [15]:
#Plot para visualizar el cambio en la distribución de la data imputada
cambio_distribucion('CAR_AGE','ambos')

8161
7098


In [16]:
#Plot para visualizar el cambio en la distribución de la data imputada
cambio_distribucion('YOJ','ambos')

8161
7098


In [24]:
#Estadidisticas basicas luego de limpieza para la data de coques
df2[df2[trg[0]]==1].describe()

,TARGET_FLAG,TARGET_AMT,AGE,BLUEBOOK,CAR_AGE,CLM_FREQ,HOMEKIDS,HOME_VAL,INCOME,KIDSDRIV,MVR_PTS,OLDCLAIM,TIF,TRAVTIME,YOJ
count,1866.0,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000,1866.000000
mean,1.0,4810.707451,43.011943,13931.329046,7.322518,1.207931,0.950161,112238.424973,49176.827974,0.260986,2.452304,5838.642551,4.803859,34.644695,10.034392
std,0.0,4076.870244,9.586658,7908.784450,5.327905,1.258818,1.195695,113716.480896,40223.998933,0.632413,2.574390,9644.339535,3.956073,14.988743,4.397731
min,1.0,30.277280,16.000000,1500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000
25%,1.0,2572.250000,36.000000,7660.000000,1.000000,0.000000,0.000000,0.000000,20318.500000,0.000000,0.000000,0.000000,1.000000,24.000000,9.000000
50%,1.0,4044.000000,42.000000,12330.000000,7.265280,1.000000,0.000000,111006.000000,43389.500000,0.000000,2.000000,2176.500000,4.000000,34.000000,11.000000
75%,1.0,5646.250000,50.000000,18850.000000,11.000000,2.000000,2.000000,196972.750000,68511.000000,0.000000,4.000000,6854.000000,7.000000,45.000000,13.000000
max,1.0,32203.362274,76.000000,42170.000000,25.000000,5.000000,5.000000,544783.000000,234825.000000,4.000000,13.000000,46652.000000,21.000000,79.000000,19.000000


## Limpieza Variables Categoricas

In [ ]:
#Las proporcones en las etiquetas de Kidsdrive eran muy pequeñas se aprupo la data por 0 y 1 cuando la data era igual o mayor a uno
df2['KIDSDRIV'] = [1 if i>=1 else i for i in df2['KIDSDRIV']]
dummies = []

#Ciclo para generar las columnas dummies sobre doda la data categorica
for i in cat:
    dummies.append(pd.get_dummies(df2[i],prefix=i))
df_dummies = pd.concat(dummies,axis=1)

In [ ]:
#Concatenado de la data dummie 
df_clean = pd.concat([df2,df_dummies],axis=1)
print(df_clean.shape)

In [ ]:
#Estadisticas basicas despues de limpieza
df_clean.describe()

In [ ]:
#Exportación del data frame despues de lipeza
name = 'clean_data.csv'
path = '../data/'
df_clean.to_csv(path+name,index=False)

In [ ]:
#Nuevo mapa de correlación de TARGET_FLAG despues de limpieza
plot_heatmap(df2,trg[0])

In [ ]:
#Nuevo mapa de correlación de TARGET_AMT despues de limpieza
plot_heatmap(df2,trg[1])